In [1]:
import numpy as np
from skopt import gp_minimize 
from tqdm import tqdm
from pak.function import *
import warnings
warnings.filterwarnings("ignore")
bounds = [(-5., 5.), ] * 4
FN = [Fn1S,Fn2S,Fn3S,Fn4S,Fn5S,Fn6S,Fn7S,Fn8S]

In [2]:
n_repeats = 10
n_calls = 400
n_init = 40

In [3]:
for i in range(5,6,1):
    results = []
    noimin = []
    obsmin = []
    premin = []
    with tqdm(total=n_repeats*n_calls,desc = "Optimizing") as pbar:
        def callback(result):
            if len(result.models)==0:
                noimin.append(result.x_iters[-1])
                obsmin.append(result.x_iters[-1])
                premin.append(result.x_iters[-1])   
            else:
                noimin.append(result.x)
                x_iters = np.array(result.x_iters)
                y_iters = np.array(result.func_vals)
                md = result.models[-1].fit(x_iters,y_iters)
                curmin = 1000000000
                for obs in x_iters:
                    pd = md.predict(obs.reshape(1,-1))
                    if pd < curmin:
                        temp = obs
                        curmin = pd
                obsmin.append(temp.tolist())
                res = gp_minimize(FN[i],bounds,n_initial_points=0,acq_func="LCB",n_calls=1,random_state=1,kappa=0,x0=result.x_iters,y0=result.func_vals)
                premin.append(res.x_iters[-1])
            pbar.update(1)
            
        results = [gp_minimize(FN[i],bounds,n_initial_points=n_init,initial_point_generator='lhs',acq_func="EI",n_calls=n_calls,random_state=n+20,callback=callback)
                for n in range(n_repeats)]
      
    for result in results:
        with open("Fn"+str(i+1)+"M.csv","a") as file:
            np.savetxt(file,result.x_iters,delimiter=',')
    
    with open("Fn"+str(i+1)+"M_noi.csv","a") as file:
        np.savetxt(file,noimin,delimiter=',')

    with open("Fn"+str(i+1)+"M_obs.csv","a") as file:
        np.savetxt(file,obsmin,delimiter=',')

    with open("Fn"+str(i+1)+"M_pre.csv","a") as file:
        np.savetxt(file,premin,delimiter=',')

Optimizing: 100%|██████████| 4000/4000 [6:31:01<00:00,  5.87s/it]   
